In [ ]:
# Merge features from various datasets, including creating new ones.
# This file currently includes merging population and population density,
# income, 

In [1]:
# Merge yelp business dataset with population & population density from simple maps dataset

import pandas as pd

yelpBus_df = pd.read_json('/Users/james/TIM147/yelp_dataset/yelp_academic_dataset_business.json',lines=True) # Import yelp business data
simple_df = pd.read_csv('/Users/james/Tim147/simplemaps_uszips_basicv1/uszips.csv',) # Import simple maps data
income_df = pd.read_csv('/Users/james/Tim147/income_dataset/kaggle_income.csv', encoding='iso-8859-1 ',)

In [2]:
yelpBus_df['postal_code'] = pd.to_numeric(yelpBus_df['postal_code'], errors='coerce') # Convert postal code from object to float64
yelpBus_df['postal_code'] = yelpBus_df['postal_code'].astype('Int64') # Convert postal code from float64 to Int64, with capital I so that it works with nan values

In [3]:
simple_df = simple_df[['zip','population','density']]
simple_df = simple_df.rename(columns = {'zip': 'postal_code', 'density': 'pop_density'})

In [4]:
merged_df = pd.merge(yelpBus_df, simple_df, on='postal_code', how='left') # Merge 

In [5]:
# Merge income on nearest latitude & longitude

import geopandas as gpd
from geopandas.tools import sjoin_nearest # Spatial join
from shapely.geometry import Point # 

income_df = income_df[['Lat','Lon','Median']]
income_df = income_df.rename(columns = {'Median': 'median_income'})

In [6]:
left_geo = [Point(xy) for xy in zip(merged_df['latitude'],merged_df['longitude'])] # Lat & lon of restaurant
right_geo = [Point(xy) for xy in zip(income_df['Lat'],income_df['Lon'])]

In [7]:
merged_gdf = gpd.GeoDataFrame(merged_df, geometry=left_geo)
income_gdf = gpd.GeoDataFrame(income_df, geometry=right_geo)

In [8]:
sjoined_gdf = sjoin_nearest(merged_gdf, income_gdf, how='left')

In [9]:
sjoined_df = sjoined_gdf.drop(['geometry','index_right','Lat','Lon'], axis=1)

In [10]:
sjoined_df.to_csv('yelp_business_plus.csv') #This is unnecessary at this point; other code will be used to save.

In [11]:
sjoined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162988 entries, 0 to 150345
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   business_id    162988 non-null  object 
 1   name           162988 non-null  object 
 2   address        162988 non-null  object 
 3   city           162988 non-null  object 
 4   state          162988 non-null  object 
 5   postal_code    157354 non-null  Int64  
 6   latitude       162988 non-null  float64
 7   longitude      162988 non-null  float64
 8   stars          162988 non-null  float64
 9   review_count   162988 non-null  int64  
 10  is_open        162988 non-null  int64  
 11  attributes     148067 non-null  object 
 12  categories     162873 non-null  object 
 13  hours          137871 non-null  object 
 14  population     156234 non-null  float64
 15  pop_density    156234 non-null  float64
 16  median_income  162988 non-null  int64  
dtypes: Int64(1), float64(5), int64(3),

In [12]:
# Create the feature for size of chain.

In [13]:
# Regularize the strings of the business names
sjoined_df['name'] = sjoined_df['name'].str.lower() # Make all lowercase
sjoined_df['name'] = sjoined_df['name'].str.strip() # Strip leading and following spaces
sjoined_df['name'] = sjoined_df['name'].str.replace('[^a-zA-Z0-9]','', regex=True)

# Calculate the number of locations for each business and create a column for it.
sjoined_df['locations'] = sjoined_df.groupby('name')['name'].transform('count')

In [ ]:
# Validated that the number of locations in the dataset is actually correct.
check_location_count = sjoined_df[sjoined_df['name'] == 'arbys']
check_location_count

In [16]:
'''Create intervals of chain size. 1-3 locations is considered not a chain
because it requires very little infrastructure and can be run by one business
owner. 4-20 locations is considered a small chain, as ACA chain restaurant regulations
begin at 20 locations. 21-50 is a medium chain, 51-100 a large, and 101+ a mega chain.
This may be biased by the fact that not all locations are included in the dataset,
especially for large and mega chains.'''

bins = [0, 3, 20, 50, 100, float('inf')]
labels = ['not', 'small', 'medium', 'large', 'mega']
sjoined_df['chain'] = pd.cut(sjoined_df['locations'], bins=bins, labels=labels, right=False)

In [17]:
# Create and normalize the one-hots for chain.
encoded_df = pd.get_dummies(sjoined_df, columns=['chain'])
encoded_df['chain_not'] = encoded_df['chain_not'].astype(int)
encoded_df['chain_small'] = encoded_df['chain_small'].astype(int)
encoded_df['chain_medium'] = encoded_df['chain_medium'].astype(int)
encoded_df['chain_large'] = encoded_df['chain_large'].astype(int)
encoded_df['chain_mega'] = encoded_df['chain_mega'].astype(int)

In [18]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162988 entries, 0 to 150345
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   business_id    162988 non-null  object 
 1   name           162988 non-null  object 
 2   address        162988 non-null  object 
 3   city           162988 non-null  object 
 4   state          162988 non-null  object 
 5   postal_code    157354 non-null  Int64  
 6   latitude       162988 non-null  float64
 7   longitude      162988 non-null  float64
 8   stars          162988 non-null  float64
 9   review_count   162988 non-null  int64  
 10  is_open        162988 non-null  int64  
 11  attributes     148067 non-null  object 
 12  categories     162873 non-null  object 
 13  hours          137871 non-null  object 
 14  population     156234 non-null  float64
 15  pop_density    156234 non-null  float64
 16  median_income  162988 non-null  int64  
 17  locations      162988 non-null  in

In [24]:
# Normalize median income, population density, population, review count, locations
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols = ['review_count', 'population', 'pop_density', 'median_income', 'locations']
encoded_df[cols] = mmscaler.fit_transform(encoded_df[cols])

In [25]:
encoded_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,population,pop_density,median_income,locations,chain_not,chain_small,chain_medium,chain_large,chain_mega
0,Pns2l4eNsfO8kk83dixA6A,abbyrappoportlaccmq,"1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,0.000264,...,None,0.307670,0.249804,0.192013,0.000000,1,0,0,0,0
1,mpf3x-BjTdTEA3yCZrAYPw,theupsstore,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,0.001322,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",0.488198,0.101925,0.164083,0.382353,0,0,0,0,1
2,tUFrWirKiKi_TAnsVWINQQ,target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,0.002248,...,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",0.404848,0.122170,0.058060,0.156010,0,0,0,0,1
3,MTSW4McQd7CbVtyjqoe9mw,sthonorepastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,0.009917,...,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",0.146637,0.700842,0.199020,0.000000,1,0,0,0,0
4,mWMc6_wTdE0EUBKIGXDVfA,perkiomenvalleybrewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,0.001058,...,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",0.050272,0.007717,0.299457,0.000000,1,0,0,0,0
